# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [15]:
# Instalamos el modelo predictivo Prophet
!python -m pip install prophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [16]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [17]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-06-21'
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-USD',start_date, today).reset_index()

btc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2723,2023-06-16,"$25,575.28","$26,463.17","$25,245.36","$26,327.46","$26,327.46",16324646965
2724,2023-06-17,"$26,328.68","$26,769.39","$26,174.49","$26,510.68","$26,510.68",11090276850
2725,2023-06-18,"$26,510.46","$26,675.93","$26,325.89","$26,336.21","$26,336.21",9565695129
2726,2023-06-19,"$26,335.44","$26,984.61","$26,312.83","$26,851.03","$26,851.03",12826986222
2727,2023-06-20,"$26,841.66","$28,388.97","$26,668.79","$28,327.49","$28,327.49",22211859147


In [18]:
# Validamos que no hay datos vacíos
btc_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [19]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = btc_df[['Date', 'Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds",
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = df.rename(columns = new_names)

In [20]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [21]:
df.tail()

,ds,y
2723,2023-06-16,"$25,575.28"
2724,2023-06-17,"$26,328.68"
2725,2023-06-18,"$26,510.46"
2726,2023-06-19,"$26,335.44"
2727,2023-06-20,"$26,841.66"


In [22]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "Evolucion del Bitcoin"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [24]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode = 'multiplicative')


In [26]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpd7sivhqt/9tnsyl5z.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpd7sivhqt/vzs_7pwj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=52843', 'data', 'file=/tmp/tmpd7sivhqt/9tnsyl5z.json', 'init=/tmp/tmpd7sivhqt/vzs_7pwj.json', 'output', 'file=/tmp/tmpd7sivhqt/prophet_modeltknc6y7w/prophet_model-20230621094512.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:45:12 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:45:16 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [27]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(365)
future.tail()

,ds
3088,2024-06-15
3089,2024-06-16
3090,2024-06-17
3091,2024-06-18
3092,2024-06-19


In [29]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-01
3,2016-01-04
4,2016-01-05
5,2016-01-06
6,2016-01-07
...,...
3086,2024-06-13
3087,2024-06-14
3090,2024-06-17
3091,2024-06-18


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [31]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [32]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2204,2024-06-13,"$-2,173.90","$-16,708.47","$11,323.81"
2205,2024-06-14,"$-2,186.91","$-16,988.84","$12,210.42"
2206,2024-06-17,"$-2,248.88","$-17,533.80","$11,274.28"
2207,2024-06-18,"$-2,263.00","$-16,715.17","$11,224.47"
2208,2024-06-19,"$-2,268.47","$-17,366.84","$11,437.65"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



**prophet** sirve para determinar predicciones a corto tiempo, en el grafico la zona donde solo aparece la linea azul corresponde a la prediccion, la nube a su alrededor es el margen de error, que como vemos a medida que se aleja éste aumenta.

In [33]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)